# [作業目標]
- 使用 Day 17 剛學到的方法, 對較完整的資料生成離散化特徵
- 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

# [作業重點]
- 仿照 Day 17 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
- 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [2]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = './'

### 之前做過的處理

In [3]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

(307511, 122)

In [4]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,M,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,F,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,M,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,F,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,M,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

## 練習時間
參考 Day 17 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [12]:
app_train['AMT_INCOME_TOTAL_cut'] = pd.cut(app_train['AMT_INCOME_TOTAL'],10)

In [13]:
app_train.AMT_INCOME_TOTAL_cut.value_counts()

(-91324.35, 11723085.0]       307508
(11723085.0, 23420520.0]           2
(105302565.0, 117000000.0]         1
(23420520.0, 35117955.0]           0
(35117955.0, 46815390.0]           0
(46815390.0, 58512825.0]           0
(58512825.0, 70210260.0]           0
(70210260.0, 81907695.0]           0
(81907695.0, 93605130.0]           0
(93605130.0, 105302565.0]          0
Name: AMT_INCOME_TOTAL_cut, dtype: int64

In [45]:
app_train_outlier = app_train.drop(app_train[app_train['AMT_INCOME_TOTAL'] > 2170260].index)

In [49]:
app_train_outlier['AMT_INCOME_TOTAL_cut'] = pd.cut(app_train_outlier['AMT_INCOME_TOTAL'],10)

In [50]:
app_train_outlier.AMT_INCOME_TOTAL_cut.value_counts()

(23515.65, 239085.0]      261908
(239085.0, 452520.0]       42467
(452520.0, 665955.0]        1719
(665955.0, 879390.0]         963
(879390.0, 1092825.0]        222
(1092825.0, 1306260.0]        89
(1306260.0, 1519695.0]        59
(1519695.0, 1733130.0]        20
(1733130.0, 1946565.0]        20
(1946565.0, 2160000.0]        10
Name: AMT_INCOME_TOTAL_cut, dtype: int64

In [51]:
app_train_outlier.groupby(['AMT_INCOME_TOTAL_cut']).mean()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_ANOM
AMT_INCOME_TOTAL_cut,,,,,,,,,,,,,,,,,,,,,
"(23515.65, 239085.0]",278157.718260,0.083480,0.096767,0.309758,0.692858,0.411748,1.389222e+05,5.489754e+05,24980.145383,4.914985e+05,...,0.000538,0.000496,0.000363,0.006405,0.006779,0.033997,0.246715,0.262022,1.881363,0.194988
"(239085.0, 452520.0]",278337.285233,0.065651,0.079780,0.505781,0.698495,0.443050,3.105623e+05,8.693104e+05,38473.802225,7.905050e+05,...,0.000942,0.000612,0.000141,0.006313,0.008027,0.036368,0.373581,0.287180,2.027640,0.096781
"(452520.0, 665955.0]",280036.497382,0.059919,0.116347,0.621291,0.712042,0.508435,5.413798e+05,1.096785e+06,47841.013089,1.009008e+06,...,0.001163,0.000000,0.000582,0.006940,0.006309,0.042271,0.510410,0.249842,1.812618,0.065154
"(665955.0, 879390.0]",275733.873313,0.047767,0.226376,0.624091,0.698858,0.508827,7.049415e+05,1.131253e+06,52480.537383,1.056134e+06,...,0.000000,0.000000,0.000000,0.009281,0.012761,0.029002,0.414153,0.238979,1.501160,0.066459
"(879390.0, 1092825.0]",272317.666667,0.054054,0.292793,0.653153,0.630631,0.459459,9.190961e+05,1.202642e+06,59227.702703,1.139959e+06,...,0.000000,0.000000,0.004505,0.000000,0.000000,0.026738,0.379679,0.197861,1.235294,0.040541
"(1092825.0, 1306260.0]",275306.022472,0.044944,0.191011,0.730337,0.606742,0.550562,1.147068e+06,1.179627e+06,57121.634831,1.092236e+06,...,0.000000,0.000000,0.000000,0.013699,0.000000,0.027397,0.452055,0.232877,1.561644,0.067416
"(1306260.0, 1519695.0]",274990.101695,0.050847,0.423729,0.745763,0.627119,0.694915,1.353423e+06,1.134958e+06,55218.508475,1.082669e+06,...,0.000000,0.000000,0.000000,0.020408,0.122449,0.020408,0.367347,0.387755,1.265306,0.000000
"(1519695.0, 1733130.0]",308808.950000,0.050000,0.100000,0.600000,0.500000,0.350000,1.577250e+06,1.203312e+06,63524.025000,1.138950e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.368421,0.631579,0.050000
"(1733130.0, 1946565.0]",269107.000000,0.100000,0.350000,0.600000,0.750000,0.550000,1.822500e+06,1.033044e+06,61143.750000,9.929250e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.222222,1.055556,0.050000


In [14]:
app_train['AMT_INCOME_TOTAL_qcut'] = pd.qcut(app_train['AMT_INCOME_TOTAL'],10)

In [18]:
app_train['AMT_INCOME_TOTAL_qcut'].value_counts()

(112500.0, 135000.0]       48849
(180000.0, 225000.0]       44809
(99000.0, 112500.0]        36907
(25649.999, 81000.0]       33391
(147150.0, 162000.0]       31120
(162000.0, 180000.0]       30704
(81000.0, 99000.0]         30280
(270000.0, 117000000.0]    27161
(225000.0, 270000.0]       19957
(135000.0, 147150.0]        4333
Name: AMT_INCOME_TOTAL_qcut, dtype: int64